# Introduction

Obesity is a condition defined by the [World Health Organization](https://www.who.int/topics/obesity/en/) as a "major risk factor for a number of chronic diseases, including diabetes, cardiovascular diseases and cancer" and a leading preventable cause of death in the United States and worldwide. It is also a condition that is seen increasingly in children. The [Youth Risk Behavior Surveillance System (YRBSS)](https://www.cdc.gov/healthyyouth/data/yrbs/overview.htm) was developed in 1990 by the Centers for Disease Control and Prevention (CDC) to study obesity and other health outcomes and associated risk factors among youth in the United States. The motivation for this project is to understand the rates of youth obesity over time and across locations and populations within the United States, and how they are correlated with certain nutritional, behavioral, institutional, and environmental risk factors. Two datasets from the CDC are considered: [Nutrition, Physical Activity, and Obesity - Youth Risk Behavior Surveillance System](https://chronicdata.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Youth-Risk/vba9-s8jp) contains information on the prevalence of youth obesity and associated risk factors gathered from the YRBSS, and [Nutrition, Physical Activity, and Obesity - Policy and Environmental Data](https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Policy-and/k8w5-7ju6) focuses on state policy and environmental factors.

# Data exploration

In [0]:
import math
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn import preprocessing
from sklearn import impute
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import io
from google.colab import files

In [0]:
obesity = pd.read_csv('./Youth_Risk_Behavior_Surveillance_System.csv')
obesity.head()

At first glance we can see that this dataset contains information from survey questions conducted across various years, states, and population groups. Our first task is to clean this dataset, making it easier to understand and work with. Many columns are identical or redundant to others, so we can drop them.

In [0]:
obesity.drop(columns=['YearEnd', 'LocationDesc', 'Datasource', 'Class',
                      'Topic', 'Data_Value_Unit', 'Data_Value_Type',
                      'Data_Value_Alt', 'Data_Value_Footnote_Symbol',
                      'Data_Value_Footnote', 'Low_Confidence_Limit',
                      'High_Confidence_Limit ', 'Total', 'Gender', 'Grade',
                      'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID',
                      'QuestionID', 'DataValueTypeID', 'LocationID',
                      'StratificationCategory1', 'StratificationCategoryId1',
                      'StratificationID1'], inplace=True)

Let's give the remaining columns more appropriate names.

In [0]:
obesity.rename(columns={'YearStart':'Year',
                        'LocationAbbr':'Location',
                        'Stratification1':'Group',
                        'Data_Value':'Value',
                        'Sample_Size':'SampleSize'}, inplace=True)
obesity.head()

From this preview, the data in the **Value** and **SampleSize** columns both appear to be numeric. Let's check to make sure.

In [0]:
obesity.Value.iloc[1]

In [0]:
obesity.SampleSize.iloc[1]

We can see that the **Value** column is truly numeric, but the **SampleSize** column contains numbers formatted as strings. Let's fix this.

In [0]:
obesity.SampleSize = obesity.SampleSize.apply(lambda x: pd.to_numeric(x.replace(',','')) if not pd.isnull(x) else x)
obesity.SampleSize.iloc[1]

The preview also makes it difficult to see what questions were asked in the surveys. Let's take a closer look.

In [0]:
obesity.Question.unique()

The survey responses describe our metrics of interest, the percent of students in grades 9-12 who have an overweight classification and the percent of students in grades 9-12 who have obesity. They also describe the prevalence of the associated risk factors. In the next section we will be focusing just on the percents of students with an overweight classification and with obesity, collectively termed the high-BMI categories.

# Question 1: How do rates of high-BMI categories vary over time and by population?

For right now, let's just consider the question of how many students are overweight or obese. Let's also focus on the nationwide data first, rather than by state.

In [0]:
us_obesity = obesity[(obesity.Location == 'US') & (obesity.Question.isin([
    'Percent of students in grades 9-12 who have obesity',
    'Percent of students in grades 9-12 who have an overweight classification']
    ))]
us_obesity = us_obesity[['Year','Group','SampleSize','Question','Value']]
us_obesity.sort_values(by=['Year','Group','Question'], inplace=True)
us_obesity.reset_index(drop=True, inplace=True)
us_obesity.head()

One problem with the data is that the definition of "overweight" is not precisely what we are interested in tracking. The [survey documentation](https://www.cdc.gov/healthyyouth/data/yrbs/pdf/2017/2017_yrbs_sadc_documentation.pdf) (pages 61-62) defines **obese** as: "students who were >= 95th percentile for body mass index, based on sex- and age-specific reference data from the 2000 CDC growth charts", but **overweight** as: "students who were >= 85th percentile but <95th percentile for body mass index, based on sex- and age-specific reference data from the 2000 CDC growth charts".

This latter metric is not so straightforward to interpet. For example, would an increase in the percentage of overweight students be due to students with BMI previously below 85th percentile gaining weight, or those with BMI previously above 95th percentile losing weight? It would be unclear. Better would be a metric that, like obesity, captures all students above a certain BMI threshold.

Therefore, let us define the metric **overweight or obese** as the percentage of students with BMI >= 85th percentile. Since the categories of overweight and obese are mutually exclusive and directly adjacent on the BMI scale, we can add those percentages to calculate our new metric. Before doing that, let's split the current data for the overweight and obese metrics into two separate columns.

In [0]:
us_obesity = us_obesity.pivot_table(index=['Year', 'Group', 'SampleSize'],
                                    columns='Question', values = 'Value')
del us_obesity.columns.name
us_obesity.reset_index(inplace=True)
us_obesity.rename(columns={
    'Percent of students in grades 9-12 who have an overweight classification':'Overweight',
    'Percent of students in grades 9-12 who have obesity':'Obese'}, inplace=True)
us_obesity.head()

Now we can create the **overweight or obese** column of data, and drop the **overweight** data that we're no longer interested in.

In [0]:
us_obesity['OverweightOrObese'] = us_obesity.Overweight+us_obesity.Obese
us_obesity.drop(columns="Overweight", inplace=True)
us_obesity.head()

Let's take a look at how the percentages of students in the U.S. that were obese and overweight or obese has changed from 2001 to 2017:

In [0]:
us_obesity_total = us_obesity[us_obesity.Group == 'Total']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))

ax1.plot(us_obesity_total.Year, us_obesity_total.Obese, linestyle='-')
ax1.scatter(us_obesity_total.Year, us_obesity_total.Obese, marker='o', linestyle='-')
ax2.plot(us_obesity_total.Year, us_obesity_total.OverweightOrObese, linestyle='-')
ax2.scatter(us_obesity_total.Year, us_obesity_total.OverweightOrObese, marker='o', linestyle='-')
ax1.margins(0.1)
ax1.set_xticks(us_obesity_total.Year.unique())
ax1.set_ylim(0, 10*math.ceil(us_obesity_total.Obese.max()/10))
ax1.set_title('Percent of students that are obese')
ax2.margins(0.1)
ax2.set_xticks(us_obesity_total.Year.unique())
ax2.set_ylim(0, 10*math.ceil(us_obesity_total.OverweightOrObese.max()/10))
ax2.set_title('Percent of students that are overweight or obese')

plt.show()

While the percentages of obese and overweight or obese students has declined at some points, overall they have increased with time. Will this be true also when we consider specific populations? It won't make much sense to compare obesity rates for population subgroups of different types, e.g. 12th graders and males. So let's do a comparison for each type of grouping: grade, gender, and race/ethnicity.

In [0]:
grade_grouping = ['9th', '10th', '11th', '12th']
gender_grouping = ['Male', 'Female']
race_ethnicity_grouping = ['Non-Hispanic White', 'Non-Hispanic Black',
    'Hispanic', 'Asian', 'American Indian/Alaska Native',
    'Hawaiian/Pacific Islander', '2 or more races']

us_obesity_by_grade = us_obesity[us_obesity.Group.isin(grade_grouping)]
us_obesity_by_gender = us_obesity[us_obesity.Group.isin(gender_grouping)]
us_obesity_by_race_ethnicity = us_obesity[us_obesity.Group.isin(race_ethnicity_grouping)]

grade_groups = us_obesity_by_grade.groupby('Group')
gender_groups = us_obesity_by_gender.groupby('Group')
race_ethnicity_groups = us_obesity_by_race_ethnicity.groupby('Group')

fig, ((grade_ax1, grade_ax2),
      (gender_ax1, gender_ax2),
      (race_ethnicity_ax1, race_ethnicity_ax2)
      ) = plt.subplots(3, 2, figsize=(15,15))

for name, group in grade_groups:
    grade_ax1.plot(group.Year, group.Obese, linestyle='-')
    grade_ax1.scatter(group.Year, group.Obese, marker='o', linestyle='-',
                      label=name)#s=group.SampleSize/250, 
    grade_ax2.plot(group.Year, group.OverweightOrObese, linestyle='-')
    grade_ax2.scatter(group.Year, group.OverweightOrObese, marker='o', linestyle='-',
                      label=name)#s=group.SampleSize/250, 
grade_ax1.margins(0.1)
grade_ax1.set_xticks(us_obesity_by_grade.Year.unique())
grade_ax1.set_ylim(0, 10*math.ceil(us_obesity_by_gender.Obese.max()/10))
grade_ax1.legend(loc='lower right')
grade_ax1.set_title('Percent of students that are obese')
grade_ax2.margins(0.1)
grade_ax2.set_xticks(us_obesity_by_grade.Year.unique())
grade_ax2.set_ylim(0, 10*math.ceil(us_obesity_by_gender.OverweightOrObese.max()/10))
grade_ax2.legend(loc='lower right')
grade_ax2.set_title('Percent of students that are overweight or obese')

for name, group in gender_groups:
    gender_ax1.plot(group.Year, group.Obese, linestyle='-')
    gender_ax1.scatter(group.Year, group.Obese, marker='o', linestyle='-',
                       label=name)#s=group.SampleSize/250, 
    gender_ax2.plot(group.Year, group.OverweightOrObese, linestyle='-')
    gender_ax2.scatter(group.Year, group.OverweightOrObese, marker='o', linestyle='-',
                       label=name)#s=group.SampleSize/250, 
gender_ax1.margins(0.1)
gender_ax1.set_xticks(us_obesity_by_gender.Year.unique())
gender_ax1.set_ylim(0, 10*math.ceil(us_obesity_by_gender.Obese.max()/10))
gender_ax1.legend(loc='lower right')
gender_ax1.set_title('Percent of students that are obese')
gender_ax2.margins(0.1)
gender_ax2.set_xticks(us_obesity_by_gender.Year.unique())
gender_ax2.set_ylim(0, 10*math.ceil(us_obesity_by_gender.OverweightOrObese.max()/10))
gender_ax2.legend(loc='lower right')
gender_ax2.set_title('Percent of students that are overweight or obese')

for name, group in race_ethnicity_groups:
    race_ethnicity_ax1.plot(group.Year, group.Obese, linestyle='-')
    race_ethnicity_ax1.scatter(group.Year, group.Obese, marker='o', linestyle='-',
                               label=name)#s=group.SampleSize/50, 
    race_ethnicity_ax2.plot(group.Year, group.OverweightOrObese, linestyle='-')
    race_ethnicity_ax2.scatter(group.Year, group.OverweightOrObese, marker='o', linestyle='-',
                               label=name)#s=group.SampleSize/50, 
race_ethnicity_ax1.margins(0.1)
race_ethnicity_ax1.set_xticks(us_obesity_by_race_ethnicity.Year.unique())
race_ethnicity_ax1.set_ylim(0, 10*math.ceil(us_obesity_by_race_ethnicity.Obese.max()/10))
race_ethnicity_ax1.legend(loc='upper left',ncol=2)
race_ethnicity_ax1.set_title('Percent of students that are obese')
race_ethnicity_ax2.margins(0.1)
race_ethnicity_ax2.set_xticks(us_obesity_by_race_ethnicity.Year.unique())
race_ethnicity_ax2.set_ylim(0, 10*math.ceil(us_obesity_by_race_ethnicity.OverweightOrObese.max()/10))
race_ethnicity_ax2.legend(loc='lower right',ncol=2)
race_ethnicity_ax2.set_title('Percent of students that are overweight or obese')

plt.show()

There isn't much that we can say about the grade subgroups. They all basically follow the overall trend, with some switching of places throughout the years. The breakdown by gender is much more stark: males have significantly higher percentages than females for all years. However, the female rates of high-BMI categories are catching up to the male rates. In fact, the increase in the percent of male students that are overweight or obese from 2001 to 2017 is negligible; the overall increase is almost entirely due to the female population.

Looking at the race/ethnicity breakdown, some populations' rates are significantly variable over time. The rates of high-BMI categories among the American Indian/Alaska Native population are the highest in 2001 but the second-lowest in 2017; meanwhile, those of the Hawaiian/Pacific Islander population are second-lowest in 2001 but highest in 2017. The order of the rates of the other five population groups are more stable. In most years, the Non-Hispanic Black and Hispanic populations have the highest rates out of these five groups, followed by 2 or more races, Non-Hispanic White, and Asian. The rates of both of the high-BMI categories for the Asian population are the lowest every year.

One thing you might notice is that the overall obesity rate peaks in 2013, yet in the race/ethnicity breakdown several populations are at or near their lowest points in that year. This is because there are simply so many more students in certain populations than others, specifically Non-Hispanic White, whose obesity rate peaked in 2013.

In [0]:
us_obesity[['Group','SampleSize']].groupby('Group').mean().loc[
    race_ethnicity_grouping].sort_values(by='SampleSize', ascending=False)

If we want to allude to the populations' differing effects on the overall trend, we can plot the data with varying marker size to illustrate populations' relative sample sizes.

In [0]:
us_obesity_by_race_ethnicity = us_obesity[us_obesity.Group.isin(race_ethnicity_grouping)]

race_ethnicity_groups = us_obesity_by_race_ethnicity.groupby('Group')

fig, ((race_ethnicity_ax1, race_ethnicity_ax2)
      ) = plt.subplots(1, 2, figsize=(15,5))

for name, group in race_ethnicity_groups:
    race_ethnicity_ax1.plot(group.Year, group.Obese, linestyle='-')
    race_ethnicity_ax1.scatter(group.Year, group.Obese, marker='o', linestyle='-',
                               s=group.SampleSize/50, label=name)
    race_ethnicity_ax2.plot(group.Year, group.OverweightOrObese, linestyle='-')
    race_ethnicity_ax2.scatter(group.Year, group.OverweightOrObese, marker='o', linestyle='-',
                               s=group.SampleSize/50, label=name)
race_ethnicity_ax1.margins(0.1)
race_ethnicity_ax1.set_xticks(us_obesity_by_race_ethnicity.Year.unique())
race_ethnicity_ax1.set_ylim(0, 10*math.ceil(us_obesity_by_race_ethnicity.Obese.max()/10))
race_ethnicity_ax1.legend(loc='upper left',ncol=2)
race_ethnicity_ax1.set_title('Percent of students that are obese')
race_ethnicity_ax2.margins(0.1)
race_ethnicity_ax2.set_xticks(us_obesity_by_race_ethnicity.Year.unique())
race_ethnicity_ax2.set_ylim(0, 10*math.ceil(us_obesity_by_race_ethnicity.OverweightOrObese.max()/10))
race_ethnicity_ax2.legend(loc='lower right',ncol=2)
race_ethnicity_ax2.set_title('Percent of students that are overweight or obese')

plt.show()

# Question 2: How do rates of high-BMI categories vary over time and by location?

Now let's take a look at how the rates of high-BMI categories vary between states. Rather than considering the variation in high-BMI rates across states and populations simultaneously, we will consider variation by state only.

In [0]:
state_obesity = obesity[~(obesity.Location == 'US') & (obesity.Question.isin([
    'Percent of students in grades 9-12 who have obesity',
    'Percent of students in grades 9-12 who have an overweight classification']
    ))]
state_obesity = state_obesity[state_obesity.Group == 'Total']

Let's split the columns by question and calculate the **overweight or obese** metric as before.

In [0]:
state_obesity = state_obesity[['Year','Location','SampleSize','Question','Value']]
state_obesity = state_obesity.pivot_table(index=['Year', 'Location', 'SampleSize'],
                                    columns='Question', values = 'Value')
del state_obesity.columns.name
state_obesity.reset_index(inplace=True)
state_obesity.rename(columns={
    'Percent of students in grades 9-12 who have an overweight classification':'Overweight',
    'Percent of students in grades 9-12 who have obesity':'Obese'}, inplace=True)
state_obesity['OverweightOrObese'] = state_obesity.Overweight+state_obesity.Obese
state_obesity.drop(columns="Overweight", inplace=True)
state_obesity.head()

Now let's try plotting.

In [0]:
state_groups = state_obesity.groupby('Location')

fig, ((state_ax1, state_ax2)) = plt.subplots(1, 2, figsize=(15,5))

for name, group in state_groups:
    state_ax1.plot(group.Year, group.Obese, linestyle='-')
    state_ax1.scatter(group.Year, group.Obese, marker='o', linestyle='-',
                      label=name)#s=group.SampleSize/250, 
    state_ax2.plot(group.Year, group.OverweightOrObese, linestyle='-')
    state_ax2.scatter(group.Year, group.OverweightOrObese, marker='o', linestyle='-',
                      label=name)#s=group.SampleSize/250, 
state_ax1.margins(0.1)
state_ax1.set_xticks(state_obesity.Year.unique())
state_ax1.set_ylim(0, 10*math.ceil(state_obesity.Obese.max()/10))
state_ax1.legend(loc='upper left',ncol=5)
state_ax1.set_title('Percent of students that are obese')
state_ax2.margins(0.1)
state_ax2.set_xticks(state_obesity.Year.unique())
state_ax2.set_ylim(0, 10*math.ceil(state_obesity.OverweightOrObese.max()/10))
state_ax2.legend(loc='upper left',ncol=5)
state_ax2.set_title('Percent of students that are overweight or obese')

plt.show()

Plotting all 51 locations (some territories are included, and some states are missing) is quite a mess. How can we make this easier to interpret? Let's look specifically at the locations with highest and lowest obesity rates.

First, let's compare each location's rates to the national average for each specific year. That way, if a location only has data for certain years it will be taken into account.

In [0]:
state_obesity_diff_from_us = (state_obesity.groupby(['Location','Year']).mean(
    )-us_obesity_total.groupby('Year').mean())[['Obese','OverweightOrObese']]
state_obesity_diff_from_us.head()

Now we can take the average across all years for which a state has data.

In [0]:
state_mean_obesity = state_obesity_diff_from_us.reset_index().groupby(
    'Location').mean()[['Obese','OverweightOrObese']]
state_mean_obesity.head()

Finally, we can sort to get the locations with highest and lowest rates of high-BMI categories.

In [0]:
lowest_obesity = list(state_mean_obesity.sort_values(by='Obese').index[0:5])
lowest_obesity

In [0]:
highest_obesity = list(state_mean_obesity.sort_values(
    by='Obese', ascending=False).index[0:5])
highest_obesity

In [0]:
lowest_ovr_or_obesity = list(state_mean_obesity.sort_values(
    by='OverweightOrObese').index[0:5])
lowest_ovr_or_obesity

In [0]:
highest_ovr_or_obesity = list(state_mean_obesity.sort_values(
    by='OverweightOrObese', ascending=False).index[0:5])
highest_ovr_or_obesity

Now we should be able to plot trends over time with more clarity.

In [0]:
fig, ((state_ax1, state_ax2)) = plt.subplots(1, 2, figsize=(15,5))

state_ax1.plot(us_obesity_total.Year, us_obesity_total.Obese, 'k', label='US')
for name, group in state_obesity[state_obesity.Location.isin(lowest_obesity)].groupby('Location'):
    state_ax1.plot(group.Year, group.Obese, '-s', label=name)
for name, group in state_obesity[state_obesity.Location.isin(highest_obesity) &
                                 state_obesity.Location.isin(highest_ovr_or_obesity)].groupby('Location'):
    state_ax1.plot(group.Year, group.Obese, '-o', label=name)
for name, group in state_obesity[state_obesity.Location.isin(highest_obesity) &
                                 ~state_obesity.Location.isin(highest_ovr_or_obesity)].groupby('Location'):
    state_ax1.plot(group.Year, group.Obese, '--o', label=name)
state_ax1.margins(0.1)
state_ax1.set_xticks(state_obesity.Year.unique())
state_ax1.set_ylim(0, 10*math.ceil(state_obesity.Obese.max()/10))
state_ax1.legend(loc='upper left',ncol=3)
state_ax1.set_title('Percent of students that are obese')

state_ax2.plot(us_obesity_total.Year, us_obesity_total.OverweightOrObese, 'k', label='US')
for name, group in state_obesity[state_obesity.Location.isin(lowest_ovr_or_obesity)].groupby('Location'):
    state_ax2.plot(group.Year, group.OverweightOrObese, '-s', label=name)
for name, group in state_obesity[state_obesity.Location.isin(highest_ovr_or_obesity) &
                                 state_obesity.Location.isin(highest_obesity)].groupby('Location'):
    state_ax2.plot(group.Year, group.OverweightOrObese, '-o', label=name)
for name, group in state_obesity[state_obesity.Location.isin(highest_ovr_or_obesity) &
                                 ~state_obesity.Location.isin(highest_obesity)].groupby('Location'):
    state_ax2.plot(group.Year, group.OverweightOrObese, '--o', label=name)
state_ax2.margins(0.1)
state_ax2.set_xticks(state_obesity.Year.unique())
state_ax2.set_ylim(0, 10*math.ceil(state_obesity.OverweightOrObese.max()/10))
state_ax2.legend(loc='upper left',ncol=3)
state_ax2.set_title('Percent of students that are overweight or obese')

plt.show()

The five locations with below-average rates of high-BMI categories were Colorado, Idaho, Montana, Utah, and Wyoming, all located in the Rocky Mountains. The five with above-average rates were Guam, Kentucky, Mississippi, Tennessee, and either Arkansas (obesity) or Louisiana (overweightness or obesity). Besides Guam, these are all located along the Mississippi River. There was a fair amount of switching places between the locations in each group, but no switching between the groups. In fact, with the exception of 2013, none of these states ever crossed the U.S. average. It appears that geographic location may be strongly correlated with youth obesity.

# Question 3: What nutritional, behavioral, institutional, and environmental factors are associated with rates of high-BMI categories?

Now we're ready to consider some of the other data from the survey responses. Let's separate the survey questions to get a better idea.

In [0]:
obesity = obesity.drop(columns='SampleSize').pivot_table(index=['Year', 'Location', 'Group'], columns='Question')
obesity.columns = obesity.columns.get_level_values(1)
del obesity.columns.name
obesity.reset_index(inplace=True)
obesity.head()

Let's give these columns shorter names and create the **overweight or obese** column.

In [0]:
obesity.rename(columns={'Percent of students in grades 9-12 watching 3 or more hours of television each school day':'TV',
                        'Percent of students in grades 9-12 who achieve 1 hour or more of moderate-and/or vigorous-intensity physical activity daily':'Exercise',
                        'Percent of students in grades 9-12 who consume fruit less than 1 time daily':'LackOfFruit',
                        'Percent of students in grades 9-12 who consume vegetables less than 1 time daily':'LackOfVeggies',
                        'Percent of students in grades 9-12 who drank regular soda/pop at least one time per day':'Soda',
                        'Percent of students in grades 9-12 who have an overweight classification':'Overweight',
                        'Percent of students in grades 9-12 who have obesity':'Obese',
                        'Percent of students in grades 9-12 who participate in daily physical education':'PhysEd'}, inplace=True)
obesity = obesity[['Year','Location','Group','TV','Exercise','PhysEd','LackOfFruit','LackOfVeggies','Soda','Overweight','Obese']]
obesity['OverweightOrObese'] = obesity.Overweight+obesity.Obese
obesity.drop(columns='Overweight', inplace=True)
obesity.head()

The columns pertaining to fruit and vegetable consumption have been named **LackOfFruit** and **LackOfVeggies** to indicate that higher data values correspond to less fruit or vegetable consumption, in contrast to the other columns.

The factors included in this dataset can be described as nutritional (**LackOfFruit**, **LackOfVeggies**, **Soda**) and behavioral (**TV**, **Exercise**, **PhysEd**). The physical education factor is institutional in nature as well. By considering a [second dataset](https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Policy-and/k8w5-7ju6) from the Division of Nutrition, Physical Activity, and Obesity, we can analyze the effects of state policy and infrastructure concerning nutrition and physical activity.

In [0]:
policy = pd.read_csv('./Policy_and_Environmental_Data.csv')
policy.head()

This dataset looks similar to the obesity dataset in that it contains the answers to various questions across states and years. One difference is the lack of a column for population groups; the data are simply statewide statistics.

Let's clean the dataset a bit.

In [0]:
policy = policy[['YearStart','LocationAbbr','Question','Data_Value']]
policy.rename(columns={'YearStart':'Year','LocationAbbr':'Location','Data_Value':'Value'}, inplace=True)
policy.sort_values(by=['Year','Location'], inplace=True)
policy.reset_index(drop=True, inplace=True)
policy.head()

We can see that this dataset contains at least one year not present in the obesity dataset. Let's remove all the rows for which either the year or location don't match any of the obesity data.

In [0]:
policy = policy[(policy.Location.isin(obesity.Location.unique())) &
                (policy.Year.isin(obesity.Year.unique()))]
policy.reset_index(drop=True, inplace=True)
policy.head()

Let's take a look at the questions in this dataset to see which we want to consider.

In [0]:
policy.Question.unique()

Some refer to breastfeeding, another aspect of health studied by the surveys. Others apply to children significantly younger than the high school students we are considering. Let's keep only the relevant questions.

In [0]:
policy = policy[policy.Question.isin([
    'Number of farmers markets per 100,000 residents',
    'State child care regulations align with national standards for limiting total media time for children 2 years or older to not more than 30 minutes once a week',
    'State child care regulations align with national standards for avoiding sugar, including concentrated sweets such as candy, sodas, sweetened drinks, fruit nectars, and flavored milk',
    'State child care regulations align with national standards for serving fruits',
    'State child care regulations align with national standards for serving vegetables',
    'State has adopted some form of a Complete Streets policy',
    'Percent of U.S. population living within 1/2 mile of a park',
    'Percent of farmers markets that accept WIC Farmers Market Nutrition Program coupons',
    'Number of food hubs in each state'])]

In [0]:
policy.Value.iloc[0]

In [0]:
policy.Value.iloc[346]

Here we are dealing with the same issue of numbers formatted as strings, but with some genuine string values that we must maintain.

In [0]:
policy.Value = policy.Value.apply(lambda x: x if x in ['No','Yes'] else pd.to_numeric(x.replace(',','')))

In [0]:
policy.Value.iloc[0]

In [0]:
policy.Value.iloc[346]

Now we can split the questions into separate columns and rename.

In [0]:
policy = policy.pivot_table(index=['Year', 'Location'], columns='Question', aggfunc='first')
policy.columns = policy.columns.get_level_values(1)
del policy.columns.name
policy.reset_index(inplace=True)
policy.rename(columns={'Number of farmers markets per 100,000 residents':'FarmMarket',
                       'Number of food hubs in each state':'FoodHubs',
                       'Percent of U.S. population living within 1/2 mile of a park':'NearPark',
                       'Percent of farmers markets that accept WIC Farmers Market Nutrition Program coupons':'WIC',
                       'State child care regulations align with national standards for avoiding sugar, including concentrated sweets such as candy, sodas, sweetened drinks, fruit nectars, and flavored milk':'SugarReg',
                       'State child care regulations align with national standards for limiting total media time for children 2 years or older to not more than 30 minutes once a week':'MediaReg',
                       'State child care regulations align with national standards for serving fruits':'FruitReg',
                       'State child care regulations align with national standards for serving vegetables':'VeggieReg',
                       'State has adopted some form of a Complete Streets policy':'CompleteStreets'}, inplace=True)
policy.head()

Clearly these columns have some missing values. But first let's merge this dataset with our obesity dataset.

In [0]:
obesity = obesity.merge(policy, left_on=['Year','Location'],
                        right_on=['Year','Location'], how='left')
obesity = obesity[['Year','Location','Group','TV','Exercise','PhysEd',
    'LackOfFruit','LackOfVeggies','Soda','FarmMarket','FoodHubs','NearPark',
    'WIC','SugarReg','MediaReg','FruitReg','VeggieReg','CompleteStreets',
    'Obese','OverweightOrObese']]
obesity.replace('No', 0, inplace=True)
obesity.replace('Yes', 1, inplace=True)
obesity.head()

We should check if any columns have all the same values after the merge.

In [0]:
obesity.describe()

The **SugarReg** and **MediaReg** columns have standard deviations of zero after merging with the obesity dataset. We will have to drop them.

In [0]:
obesity.drop(columns=['SugarReg','MediaReg'], inplace=True)

Now let's take a look at the mean values of these factors within each location.

In [0]:
mean_values_by_location = obesity[obesity.Group == 'Total'].drop(
    columns=['Year','Group','Obese','OverweightOrObese']).groupby('Location').mean()
mean_values_by_location.head()

The list of locations lowest in obesity and the list of locations lowest in overweightness or obesity are identical, so we can use either one.

In [0]:
lowest_BMI = lowest_obesity.copy()
lowest_BMI_mean_values = mean_values_by_location.loc[lowest_BMI].mean()
lowest_BMI_mean_values

For the locations with the highest BMI categories, we must combine the two lists.

In [0]:
highest_BMI = highest_obesity.copy()
highest_BMI.extend(highest_ovr_or_obesity)
highest_BMI = list(np.unique(highest_BMI))
highest_BMI_mean_values = mean_values_by_location.loc[highest_BMI].mean()
highest_BMI_mean_values

Let's take a look at the U.S. average values as well.

In [0]:
us_avg_values = mean_values_by_location.loc['US']
us_avg_values

The **FoodHubs**, **FruitReg**, **VeggieReg**, and **CompleteStreets** factors represent amounts rather than percentages, so the data labeled 'US' is giving us a sum rather than an average. We can replace these incorrect numbers with averages taken across all states. The reason we did not just do this originally for all factors is that for those factors representing percentages, we do not want the simple average of the location data. We instead want the national average, which would be equal to the average of the location data weighted by location population per year. Calculating it this way would be far too tedious when we can simply use the data labeled 'US'.

In [0]:
us_avg_values.loc[['FoodHubs','FruitReg','VeggieReg',
    'CompleteStreets']] = mean_values_by_location.mean().loc[['FoodHubs',
    'FruitReg','VeggieReg','CompleteStreets']]
us_avg_values

Now we can plot how each factor differs between the lowest BMI locations, U.S. average, and the highest BMI locations.

In [0]:
labels = mean_values_by_location.columns

x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(18,4))
rects1 = ax.bar(x-width, lowest_BMI_mean_values, width, color = '#3377EE',
                label='Average for low-BMI locations')
rects2 = ax.bar(x, us_avg_values, width, color = '#FFDD00',
                label='U.S. average')
rects3 = ax.bar(x+width, highest_BMI_mean_values, width, color = '#FF4433',
                label='Average for high-BMI locations')

ax.set_ylabel(' ')
ax.set_title('Metrics for high- and low-BMI locations vs. U.S. average')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
plt.show()

In order to better visualize all factors, we can normalize the data within each.

In [0]:
mean_values = pd.concat([lowest_BMI_mean_values, us_avg_values, highest_BMI_mean_values], axis=1)
mean_values.columns = ['LowBMI', 'avg', 'HighBMI']
lowest_BMI_normalized = mean_values.LowBMI/mean_values.max(axis=1)
avg_BMI_normalized = mean_values.avg/mean_values.max(axis=1)
highest_BMI_normalized = mean_values.HighBMI/mean_values.max(axis=1)

labels = mean_values_by_location.columns

x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(18,4))
rects1 = ax.bar(x-width, lowest_BMI_normalized, width, color = '#3377EE',
                label='Average for low-BMI locations')
rects2 = ax.bar(x, avg_BMI_normalized, width, color = '#FFDD00',
                label='U.S. average')
rects3 = ax.bar(x+width, highest_BMI_normalized, width, color = '#FF4433',
                label='Average for high-BMI locations')

ax.set_ylabel(' ')
ax.set_title('Metrics for high- and low-BMI locations vs. U.S. average')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylim(top=1.4)
ax.legend()
plt.show()

Low-BMI locations have percentages lower than the national average of high-school students that "watch 3 or more hours of television each school day", "consume vegetables less than 1 time daily", and "drink regular soda/pop at least one time per day". Meanwhile, high-BMI locations have percentages higher than the national average for these same categories, as well as for students "consuming fruit less than 1 time daily".

Low-BMI locations also have more farmers markets per capita and more people living within half a mile of a park than the national average, while high-BMI locations have fewer of these than the national average. Interestingly, none of the low-BMI locations during any of the years in this data have "state child care regulations that align with national standards for serving fruits" or likewise for vegetables. High-BMI locations have these regulations for serving vegetables at rates higher than average, but not for fruits. Perhaps the high-BMI locations have attempted to address their rates of high-BMI categories among students through regulations, while the low-BMI locations have not seen the need.

Comparing these factors among three groups of locations is not the only way to consider their correlation with the prevalence of high-BMI categories. We can calculate the Pearson correlation coefficient between all columns in our data., and visualize the correlations between the high-BMI metrics and associated factors.

In [0]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(8,7))

ax1.set_title('Correlation across location')
df = obesity[~(obesity.Location == 'US') & (obesity.Group == 'Total')]
heatmap(df.corr().drop(columns=['Year','Obese','OverweightOrObese']
    ).loc[['Obese','OverweightOrObese']], vmin=-1, vmax=1, annot=True, fmt=".2f",
    cmap='RdBu_r', cbar=False, ax=ax1)

ax2.set_title('Correlation across location and school grade')
df = obesity[~(obesity.Location == 'US') & (obesity.Group.isin(grade_grouping))]
heatmap(df.corr().drop(columns=['Year','Obese','OverweightOrObese']
    ).loc[['Obese','OverweightOrObese']], vmin=-1, vmax=1, annot=True, fmt=".2f",
    cmap='RdBu_r', cbar=False, ax=ax2)

ax3.set_title('Correlation across location and gender')
df = obesity[~(obesity.Location == 'US') & (obesity.Group.isin(gender_grouping))]
heatmap(df.corr().drop(columns=['Year','Obese','OverweightOrObese']
    ).loc[['Obese','OverweightOrObese']], vmin=-1, vmax=1, annot=True, fmt=".2f",
    cmap='RdBu_r', cbar=False, ax=ax3)

ax4.set_title('Correlation across location and race/ethnicity')
df = obesity[~(obesity.Location == 'US') & (obesity.Group.isin(race_ethnicity_grouping))]
heatmap(df.corr().drop(columns=['Year','Obese','OverweightOrObese']
    ).loc[['Obese','OverweightOrObese']], vmin=-1, vmax=1, annot=True, fmt=".2f",
    cmap='RdBu_r', cbar=False, ax=ax4)

plt.setp(ax1.get_yticklabels(), rotation=0)
plt.setp(ax2.get_yticklabels(), rotation=0)
plt.setp(ax3.get_yticklabels(), rotation=0)
plt.setp(ax4.get_yticklabels(), rotation=0)
plt.tight_layout(1.5)
plt.show()

The relative importance of factors mostly matches what we saw in the bar chart. Watching TV, drinking soda, eating fruits less often, and especially eating vegetables less often are positively correlated with higher rates of high-BMI across the statewide data, grade-grouped data, gender-grouped data, and race/ethnicity-grouped data. Living near a park is negatively correlated with higher rates of high-BMI across these data groups. However, the numbers of farmers markets per capita and the prevalence of vegetable regulations do not appear significant from this analysis, in contrast to the bar chart.

Also, it is important to remember that for the policy/environmental factors, we do not have population-specific data. In the correlation across location and race/ethnicity for example, we are looking at the correlation between obesity rates in different racial/ethnic groups across many locations versus the percent of all students in that location, regardless of racial/ethnic group, living within half a mile of a park. With more detailed data we would be able to further refine this analysis and consider more specific questions.

# Question 4: Can we predict rates of high-BMI categories?

The correlation calculation we performed quantifies the degree to which there is a linear relationship between one of our high-BMI metrics of interest and another factor in the data. However, it does not take into account the other factors. To do that we can train a linear regression model on the data and see what factors are most important for model performance.

Let's start with the data across all locations and drop the columns we don't want to be in the model.

In [0]:
df = obesity[~(obesity.Location == 'US') & (obesity.Group == 'Total')]
df.drop(columns=['Year','Location','Group','OverweightOrObese'], inplace=True)
df.head()

There are a lot of missing values which we must handle before building a model. A common strategy is to impute (fill) the missing values with the a single value based on the statistics of that column. Let's do this with the mode (the cmost frequent value).

In [0]:
imp = impute.SimpleImputer(strategy='most_frequent')
df[df.columns] = imp.fit_transform(df[df.columns])
df.head()

Now we will split the data into a training set and a test set, build a linear regression model, train it, and use it to make predictions of obesity rates based on the data.

In [0]:
X = df.drop('Obese', axis=1)
y = df['Obese']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_train_preds = model.predict(X_train)
y_test_preds = model.predict(X_test)

We can use the describe function from SciPy to look at the minimum and maximum of the training set and test set predictions. The percent of students that are obese can never be less than 0 or greater than 100. If any predictions fall outside these boundaries, or even if they are close, then linear regression could be a bad choice.

In [0]:
stats.describe(y_train_preds)[1]

In [0]:
stats.describe(y_test_preds)[1]

Not only are these percentages possible, they seem reasonable based on what we know about the data. Now we can quantify the accuracy of our model using the coefficient of determination between our predictions and the correct predictions.

In [0]:
r2_score(y_train, y_train_preds)

In [0]:
r2_score(y_test, y_test_preds)

Model accuracy is only 53.5% for the training set. So even when considering all factors at once and being given all the right answers, a linear model can only achieve so much. The data is far from perfectly correlated.

The model accuracy for the test set is 48.4%, not far behind the training score. The model is generalizing fairly well to new samples, despite both scores being lower than we might like.

Now let's look at which factors the model weights most heavily.

In [0]:
pd.DataFrame(index=X.columns, data=model.coef_, columns = ['coef']
             ).sort_values(by='coef', ascending=False)

Negative values indicate that the model predicts lower obesity when the factor value is high, while for positive coefficients it predicts higher rates. The absolute value of the coefficient represents the degree of influence of that factor on model predictions. Currently **VeggieReg** and **CompleteStreets** are being considered most heavily by the model. From the bar chart we saw that these factors, especially **VeggieReg**, seem to be correlated with geographic lcoation.

Will model performance improve if we consider geographic location? Rather than training the model on the specific state or territory, let's take a broader approach. We can use the four regions of the country defined by the [U.S. Census Bureau](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf): Northeast, Midwest, South, and West. Territories are not classified in a region, so we will group them in a separate class.

In [0]:
west = ['AK', 'HI', 'WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM']
midwest = ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH']
south = ['TX', 'OK', 'AR', 'LA', 'KY', 'TN', 'MS', 'AL', 'WV', 'MD', 'DE', 'DC', 'VA', 'NC', 'SC', 'GA', 'FL']
northeast = ['PA', 'NJ', 'NY', 'CT', 'RI', 'MA', 'VT', 'NH', 'ME']
territories = ['VI', 'GU', 'PR']

obesity_with_region = obesity.copy()
obesity_with_region['Region'] = obesity_with_region.Location.apply(
    lambda x: 'West' if x in west else 'Midwest' if x in midwest else 'South'
    if x in south else 'Northeast' if x in northeast else 'Territories'
    if x in territories else 'Other')
obesity_with_region.head()

We'll process the data as before, but with an additional step: encoding our new categorical variable numerically.

In [0]:
df = obesity_with_region[~(obesity_with_region.Location == 'US') &
                         (obesity_with_region.Group == 'Total')]
df.drop(columns=['Year','Location','Group','OverweightOrObese'], inplace=True)
df = pd.get_dummies(df, drop_first=True)
df.head()

Now we have four columns of zeros and ones to tell us whether or not a location is in the Northeast, South, territories, or West. Zeros in all four columns mean that it is in the Midwest.

In [0]:
imp = impute.SimpleImputer(strategy='most_frequent')
df[df.columns] = imp.fit_transform(df[df.columns])
X = df.drop('Obese', axis=1)
y = df['Obese']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_train_preds = model.predict(X_train)
y_test_preds = model.predict(X_test)
r2_score(y_train, y_train_preds)

In [0]:
r2_score(y_test, y_test_preds)

By simply telling the model to which of these five regions each sample belongs, model accuracy has improved. Let's zoom in on the location a bit more by providing the region subdivisions used by the Census Bureau.

In [0]:
NewEngland = ['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
MidAtlantic = ['NJ', 'NY', 'PA']
EastNorthCentral = ['IL', 'IN', 'MI', 'OH', 'WI']
WestNorthCentral = ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']
SouthAtlantic = ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV']
EastSouthCentral = ['AL', 'KY', 'MS', 'TN']
WestSouthCentral = ['AR', 'LA', 'OK', 'TX']
Mountain = ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY']
Pacific = ['AK', 'CA', 'HI', 'OR', 'WA']

obesity_with_division = obesity.copy()
obesity_with_division['Division'] = obesity_with_division.Location.apply(
    lambda x: 'NewEngland' if x in NewEngland else 'MidAtlantic'
    if x in MidAtlantic else 'EastNorthCentral' if x in EastNorthCentral else
    'WestNorthCentral' if x in WestNorthCentral else 'SouthAtlantic'
    if x in SouthAtlantic else 'EastSouthCentral' if x in EastSouthCentral else
    'WestSouthCentral' if x in WestSouthCentral else 'Mountain'
    if x in Mountain else 'Pacific' if x in Pacific else 'Territories'
    if x in territories else 'Other')

df = obesity_with_division[~(obesity_with_division.Location == 'US') &
                         (obesity_with_division.Group == 'Total')]
df.drop(columns=['Year','Location','Group','OverweightOrObese'], inplace=True)
df = pd.get_dummies(df, drop_first=True)
imp = impute.SimpleImputer(strategy='most_frequent')
df[df.columns] = imp.fit_transform(df[df.columns])
X = df.drop('Obese', axis=1)
y = df['Obese']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_train_preds = model.predict(X_train)
y_test_preds = model.predict(X_test)
r2_score(y_train, y_train_preds)

In [0]:
r2_score(y_test, y_test_preds)

We've managed to get test accuracy above 60%; still not ideal for a useful predictive model. Let's consider some other algorithms.

In [0]:
def train_obesity_model(df, response_col, cols_to_drop, test_size, model_type):
    '''
    INPUT:
    df - pandas dataframe with obesity data or a subset thereof
    response_col - column of data to predict
    cols_to_drop - columns to drop and not train on
    test_size - proportion of the test-train split that is for testing, between 0 and 1
    model_type - choice of sklearn algorithm: 'LinearRegression', 'Lasso', 'Ridge',
        'ElasticNet', 'SVR', 'Bagging', 'RandomForest', 'AdaBoost'
    impute_strategy - how to impute missing values: 'mean', 'median', 'most_frequent'
    scaling - Boolean, apply min-max scaling if True or no scaling if False

    STEPS:
    Drop the rows with missing values for the response column
    Drop columns with all NaN values
    Drop the columns on which we do not want to train, such as the location or the other response column
    Split categorical variables using get_dummies()
    Impute missing values with the mean
    Split into explanatory and response variables
    Split into training and test sets
    Predict the response variable
    Score the model

    OUTPUT:
    a pandas Series that contains the training score and test score
    '''
    
    # Drop the rows with missing response values
    df = df.dropna(subset=[response_col], axis=0)

    # Drop columns with all NaN values
    df = df.dropna(how='all', axis=1)

    # Drop the columns on which we do not want to train, including the other response column
    df.drop(columns=cols_to_drop, inplace=True)

    # Get dummies
    df = pd.get_dummies(df, drop_first=True)

    # Impute missing values
    imp = impute.SimpleImputer(strategy='most_frequent')
    df[df.columns] = imp.fit_transform(df[df.columns])

    # Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Build model
    if model_type == 'LinearRegression':
      model = linear_model.LinearRegression()
    elif model_type == 'Lasso':
      model = linear_model.Lasso()
    elif model_type == 'Ridge':
      model = linear_model.Ridge()
    elif model_type == 'ElasticNet':
      model = linear_model.ElasticNet()
    elif model_type == 'SVR':
      model = svm.SVR(kernel='rbf', C=40)

    # Fit model to training data
    model.fit(X_train, y_train)

    # Predict for training and test data
    y_train_preds = model.predict(X_train)
    y_test_preds = model.predict(X_test)

    # Score model on training and test data
    training_score = r2_score(y_train, y_train_preds)
    test_score = r2_score(y_test, y_test_preds)

    # Return scores
    score = pd.Series({'training_score':training_score, 'test_score':test_score})

    return score

This function perfoms the steps of data preparation, model building, training, predicition, and scoring for five different choices of algorithm: linear regression, lasso regression, ridge regression, elastic net regression, and support vector regression (SVR). Lasso and ridge are regularization techniques that perform linear regression but with penalties imposed on coefficients being too high. They use two different penalty functions. Elastic net combines these two regularization methods in an attempt to overcome their limitations. SVR is an extension of support vector machines, originally used in classification problems. Let's build another function to plot model accuracy on various subsets of the data using these five algorithms.

In [0]:
def plot_alg_performance(df, response_col):
    scores = pd.DataFrame({'LinearRegression':[0]*4, 'Lasso':[0]*4,
        'Ridge':[0]*4, 'ElasticNet':[0]*4, 'SVR':[0]*4}, index=[
        '["Total"]', 'grade_grouping', 'gender_grouping',
        'race_ethnicity_grouping'])

    other_col = [x for x in ['Obese','OverweightOrObese'] if not x == response_col][0]
    cols_to_drop = ['Year','Location','Group']
    cols_to_drop.append(other_col)

    for grouping in scores.index:
      df_grouping = eval('df[~(df.Location == "US") & (df.Group.isin('+grouping+'))]')
      for model_type in scores.columns:
        score = train_obesity_model(df_grouping, response_col, cols_to_drop,
            test_size=.3, model_type = model_type)
        scores.loc[grouping, model_type] = score.test_score

    labels = ['Data across location', 'Data across location and school grade',
        'Data across location and gender', 'Data across location and race/ethnicity']

    x = np.arange(len(labels))
    width = 0.15

    fig, ax = plt.subplots(figsize=(18,4))
    rects1 = ax.bar(x-2*width, scores['LinearRegression'], width, color = '#3377EE',
                    label='Linear regression')
    rects2 = ax.bar(x-width, scores['Lasso'], width, color = '#FFDD00',
                    label='Lasso regression')
    rects3 = ax.bar(x, scores['Ridge'], width, color = '#FF4433',
                    label='Ridge regression')
    rects4 = ax.bar(x+width, scores['ElasticNet'], width, color = '#55DD55',
                    label='ElasticNet')
    rects5 = ax.bar(x+2*width, scores['SVR'], width, color = '#AA55BB',
                    label='SVR')

    ax.set_ylabel(' ')
    if response_col == 'Obese':
      ax.set_title('Test score for predicting the percent of students obese')
    elif response_col == 'OverweightOrObese':
      ax.set_title('Test score for predicting the percent of students overweight or obese')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylim((0,1.2))
    ax.legend()
    ax.annotate('Max test score: '+str(round(scores.max().max(), 2)),
                (1.25, 1.05*scores.max().max()))
    ax.axhline(y=scores.max().max(), xmin=0, xmax=6, linestyle='--', c='r')
    plt.show()

In [0]:
plot_alg_performance(obesity, 'Obese')
plot_alg_performance(obesity, 'OverweightOrObese')

For both the percent of students obese and those overweight or obese, support vector regression outperforms the other algorithms. We can also see that predictive performance is better across all algorithms for the overall data by location. This may be because there are simply more data samples, which allow for more training, more learning, and thus higher accuracy when generalizing to the test samples.

In [0]:
plot_alg_performance(obesity_with_region, 'Obese')
plot_alg_performance(obesity_with_region, 'OverweightOrObese')

In [0]:
plot_alg_performance(obesity_with_division, 'Obese')
plot_alg_performance(obesity_with_division, 'OverweightOrObese')

The dominance of SVR is less when region or division are included; ridge regression and plain linear regression also perform well. Generally in data science the quality of the data is more important than the choice of algorithm. In order to have more faith in the questions we can answer from this dataset, we might want more data samples, or more features on which to train.



# Summary

In examining this data we have seen that the percents of high-school students in the United States that are obese and those that are overweight or obese have increased from 2001 to 2017, are higher in certain racial/ethnic groups, and are higher in males than females, although females are catching up. We have also learned that five states in the Rocky Mountains have the lowest rates of these high-BMI categories, and in those states students tend to watch less TV, eat more vegetables, and drink less soda than the national average. Those states also have more farmers markets per capita and more people living within half a mile of a park than the national average. Five states along the Mississippi River and Guam have the highest rates of these high-BMI categories, and students in those locations tend to watch more TV, eat fewer fruits and vegetables, and drink more soda than the national average. Those locations also have fewer farmers markets per capita and fewer people living within half a mile of a park than the national average. Similar results are seen in the single-factor correlation coefficients. We have also shown that by providing some level of information on geographic location, the test set accuracy of a linear model can be improved.

One important caveat is that BMI is not the same as body fat percentage, nor is it perfectly correlated. The metric has been criticized for its bias across height ranges (being underpredicted for shorter individuals and overpredicted for taller) and failure to account for body composition properties, such as a person's degree of musculature.

Another is that much of the data considered here are from self-report surveys, including a student's height and weight, which are used to calculate BMI. The [YRBSS methodology documentation](https://www.cdc.gov/mmwr/pdf/rr/rr6201.pdf) mentions a CDC study on the reliability of these two question in particular, in which students' heights and weights were measured after having taken the questionnaire: "Self-reported height, weight, and BMI calculated from these values were substantially reliable, but on average, students in the study overreported their height by 2.7 inches and underreported their weight by 3.5 pounds, which indicates that YRBSS probably underestimates the prevalence of overweight and obesity in adolescent populations." In addition, the policy data comes from aggregating various state and local level reports under a common standard, for example that the "state has adopted some form of a Complete Streets policy".

Finally, the factors considered here are far from the only ones that could be used to understand and predict rates of youth obesity. The documentation states that the "CDC decided that the system should focus almost exclusively on health-risk behaviors rather than on the determinants of these behaviors (e.g., knowledge, attitudes, beliefs, and skills), because there is a more direct connection between specific health-risk behaviors and specific health outcomes than between determinants of behaviors and health outcomes." However, research and analysis of the determining factors of risk behaviors is undoubtedly important in the effort to address this issue.